In [35]:
from napari import Viewer, run
from napari.layers import Image, Layer

from qtpy.QtWidgets import QWidget, QVBoxLayout, QLabel, QComboBox, QPushButton

from magicgui.widgets import create_widget, Label

from matplotlib.backends.backend_qtagg import (
    FigureCanvas, NavigationToolbar2QT as NavigationToolbar)
from matplotlib.figure import Figure


class SamplePlugin(QWidget):
    def __init__(self, viewer: Viewer):
        super().__init__()
        self.viewer = viewer
        self.select_image = create_widget(label="BF", annotation=Image)
        self.select_mask = create_widget(label="mask", annotation=Layer)
        self.canvas = FigureCanvas(Figure(figsize=(5,5)))
        self.ax = self.canvas.figure.subplots()

        self.ax.plot([0, 5, 6], [2, 7, 14])
        
        self.dims = []
        
        ll = QLabel()
        ll.setText('BF')
        self.mm = QLabel()
        self.mm.setText('mask')
        
        self.btn = QPushButton("Add operation")
        self.btn.clicked.connect(self.add_op)
        
        self.layout = QVBoxLayout()
        self.layout.addWidget(ll)
        self.layout.addWidget(self.select_image.native)
        self.layout.addStrut(1)
        self.layout.addWidget(self.mm)
        # layout.addWidget(self.select_mask.native)
        # layout.addWidget(self.canvas)

        self.setLayout(self.layout)
    
    def reset_choices(self, event=None):
        self.select_image.reset_choices(event)
        self.select_mask.reset_choices(event)
        
        self.dims.append(viewer.layers[self.select_image.current_choice].data.shape)
        
        self.mm.setText(str(self.dims[-1]))
        
        self.combo = QComboBox()
        self.combo.addItems(['max_projection', 'split_dims', 'export'])
        self.combo.currentIndexChanged.connect(self.choose_op)
        self.layout.addWidget(self.combo)
        self.layout.addWidget(self.btn)
    
    def choose_op(self, i):
        print(f'{i} selected: {self.combo.currentText()}')
    
    def add_op(self):
        op = QLabel()
        op.setText(self.combo.currentText())
        self.layout.addWidget(op)
        
        combo = QComboBox()
        combo.addItems(map(str, self.dims[-1]))
        self.layout.addWidget(combo)
        
        qdim = QLabel()
        self.layout.addWidget(qdim)
        self.dims.append(rm_dim(self.dims[-1], combo.currentText()))
        combo.currentIndexChanged.connect(lambda: qdim.setText('Output shape: ' + str(self.dims[-1])))
        
    



viewer = Viewer()

plugin = SamplePlugin(viewer)

viewer.window.add_dock_widget(plugin)

run()

/home/aaristov/miniconda3/envs/nd2/lib/python3.8/site-packages/napari_tools_menu/__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


opening /home/aaristov/Anchor/Lena/Data/E.coli/Ciprofloxacin/20220827-ecoli-cipro/day1/TRITC.nd2
{'P': 6, 'Z': 31, 'Y': 7138, 'X': 22543}
No channels, {'P': 6, 'Z': 31, 'Y': 7138, 'X': 22543}
[6, 31, 7138, 22543] 6


In [33]:
viewer.layers[plugin.select_image.current_choice].data

,Array,Chunk
Bytes,55.75 GiB,306.92 MiB
Shape,"(6, 31, 7138, 22543)","(1, 1, 7138, 22543)"
Count,372 Tasks,186 Chunks
Type,uint16,numpy.ndarray


In [32]:
ll=[6, 31, 7138, 22543]
ll.remove(6)
ll

[31, 7138, 22543]